<a href="https://colab.research.google.com/github/BastLast/DeepLearning/blob/main/image_recovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coucou

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import models

import time

In [ ]:
class CustomModel(torch.nn.Module):

  def __init__(self):
    super(LeNet5, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5), stride=1, padding=2, bias=False)
    self.relu1 = torch.nn.ReLU()
    self.avgpool1 = torch.nn.AvgPool2d(2, stride=2)
    # initialize second set of CONV => RELU => POOL layers
    self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1, padding=0, bias=False)
    self.relu2 = torch.nn.ReLU()
    self.avgpool2 = torch.nn.AvgPool2d(2, stride=2)
    # initialize third set of CONV => RELU
    self.conv3 = torch.nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=1, padding=0, bias=False)
    self.relu3 = torch.nn.ReLU()
    # initialize first set of linear => RELU layers
    self.flatten = torch.nn.Flatten()
    self.f1 = torch.nn.Linear(in_features=120, out_features=84)
    self.relu4 = torch.nn.ReLU()
    # initialize second set of linear
    self.f2 = torch.nn.Linear(in_features=84, out_features=10)
    pass
  
  def forward(self, x):
    """
      :param: x : the image to process ( shape [batch_size, 1, height, width] )
      :return: logits: the unscaled output scores that will be process by the softmax ( shape [batch_size, D_out] )
    """
    # pass the input through our first set of CONV => RELU =>
		# POOL layers
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.avgpool1(x)
    # pass the output from the previous layer through the second
    # set of CONV => RELU => POOL layers
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.avgpool2(x)
    # pass the output from the previous layer through the third
    # set of CONV => RELU => POOL layers
    x = self.conv3(x)
    x = self.relu3(x)
    # flatten the output from the previous layer and pass it
    # through our first set of linear
    x = self.flatten(x)
    x = self.f1(x)
    x = self.relu4(x)
    # pass the output to our second set of linear
    # predictions
    x = self.f2(x)
    # return the output predictions
    return x